<a href="https://colab.research.google.com/github/AlexeyUdod/aml/blob/master/dll_6_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Задание 1:


Сгенерировать последовательности, которые бы состояли из цифр (от 0 до 9)
и задавались следующим образом:
x - последовательность цифр
y1 = x1, y(i) = x(i) + x(1). Если y(i) >= 10, то y(i) = y(i) - 10

Задача:
научить модель предсказывать y(i) по x(i)
пробовать RNN, LSTM, GRU


In [1]:
import torch as tr
import copy
import time
import torch
from torch import nn
import re
import random
import tqdm

In [8]:
dev = tr.device('cuda' if tr.cuda.is_available() else 'cpu')
if dev.type == 'cuda': print('work on', tr.cuda.get_device_name(0))
else: print('work on CPU')

work on Tesla P100-PCIE-16GB


In [37]:
def f(x): return (x + x[:,0:1]) % 10
t = tr.tensor
res = []

In [5]:
class rnn1(tr.nn.Module):
    def __init__(self, n_classes, embedding_size, hidden_size, num_layers, rnn_type):
        super().__init__()
        self.emb = tr.nn.Embedding(n_classes, embedding_size)
        self.rnn = rnn_type(embedding_size, hidden_size, num_layers, batch_first = True)
        self.lin = tr.nn.Linear(hidden_size, n_classes)
    def forward(self, input):
        r = self.emb(input)
        r = self.rnn(r)[0]
        r = self.lin(r).squeeze()
        return r

In [38]:
n_iters = 100000
n_classes = 10
embedding_size = 10
hidden_size = 50
num_layers = 1
batch_size = 100
seq_len = 100
loss_best = 10**10
rnn_list = [tr.nn.RNN, tr.nn.LSTM, tr.nn.GRU]

for rnn_type in rnn_list:
    model = rnn1(n_classes, embedding_size, hidden_size, num_layers, rnn_type).to(dev)
    opt = tr.optim.Adam(model.parameters())
    crit = tr.nn.CrossEntropyLoss()

    for iter in range(n_iters):
        model.train()
        x = tr.randint(n_classes, (batch_size, seq_len)).to(dev)
        y = f(x)
        opt.zero_grad()
        pred = model(x)
        score = (y == pred.argmax(dim=-1)).sum() / t(y.shape).prod()
        if score == 1:
            res.append(f'{rnn_type.__name__} finished on iter {iter}, loss = {loss}, score = {score}')
            print(f'\n{res[-1]}\n')
            break
        loss = crit(pred.reshape(-1, n_classes), y.flatten()).to(dev)  #
        if loss < loss_best: 
            model_best = copy.copy(model)
            loss_best = loss
        loss.backward()
        opt.step()
        if iter % (n_iters / 10) == 0:
            model.eval()
            x = tr.randint(n_classes, (batch_size, seq_len)).to(dev)
            y = f(x)
            pred = model(x).argmax(dim=-1)
            score = (y == pred).sum() / t(y.shape).prod()
            better_rand = score / n_classes * 100
            print(f'{rnn_type.__name__} iter {iter}, loss = {loss}, loss_best = {loss_best}, score = {score}, better_rand = {better_rand}')

RNN iter 0, loss = 2.3158771991729736, loss_best = 2.3158771991729736, score = 0.10759999603033066, better_rand = 1.0759999752044678
RNN iter 10000, loss = 1.4611923694610596, loss_best = 1.1661938428878784, score = 0.29159998893737793, better_rand = 2.9159998893737793
RNN iter 20000, loss = 0.7689993381500244, loss_best = 0.6838308572769165, score = 0.48829999566078186, better_rand = 4.882999897003174
RNN iter 30000, loss = 0.4169624149799347, loss_best = 0.3123108744621277, score = 0.7290999889373779, better_rand = 7.2910003662109375

RNN finished on iter 36184, loss = 0.019074611365795135, score = 1.0

LSTM iter 0, loss = 2.306781530380249, loss_best = 0.01714378595352173, score = 0.10319999605417252, better_rand = 1.031999945640564

LSTM finished on iter 9272, loss = 0.016132427379488945, score = 1.0

GRU iter 0, loss = 2.3109846115112305, loss_best = 0.01445137057453394, score = 0.09829999506473541, better_rand = 0.9829999804496765

GRU finished on iter 2020, loss = 0.044665466994

In [45]:
res

['RNN finished on iter 36184, loss = 0.019074611365795135, score = 1.0',
 'LSTM finished on iter 9272, loss = 0.016132427379488945, score = 1.0',
 'GRU finished on iter 2020, loss = 0.044665466994047165, score = 1.0']

###Задание 2:

Применить LSTM для решения лекционного практического задания
http://colab.research.google.com/drive/1_rNrPHl6sYHNp-xo2G6I_SmRGTSSeICL


In [2]:
!wget https://s3.amazonaws.com/text-datasets/nietzsche.txt

--2021-04-22 22:39:22--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.145.165
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.145.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘nietzsche.txt’

nietzsche.txt       100%[===================>] 586.82K  1.34MB/s    in 0.4s    

2021-04-22 22:39:23 (1.34 MB/s) - ‘nietzsche.txt’ saved [600901/600901]



In [3]:
with open('nietzsche.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('length:', len(text))
text = re.sub('[^a-z ]', ' ', text)
text = re.sub('\s+', ' ', text)

length: 600893


In [4]:
INDEX_TO_CHAR = sorted(list(set(text)))
CHAR_TO_INDEX = {c: i for i, c in enumerate(INDEX_TO_CHAR)}
MAX_LEN = 40
STEP = 3
SENTENCES = []
NEXT_CHARS = []
for i in range(0, len(text) - MAX_LEN, STEP):
    SENTENCES.append(text[i: i + MAX_LEN])
    NEXT_CHARS.append(text[i + MAX_LEN])
print('Num sents:', len(SENTENCES))

X = torch.zeros((len(SENTENCES), MAX_LEN), dtype=int)
Y = torch.zeros((len(SENTENCES)), dtype=int)
for i, sentence in enumerate(SENTENCES):
    for t, char in enumerate(sentence):
        X[i, t] = CHAR_TO_INDEX[char]
    Y[i] = CHAR_TO_INDEX[NEXT_CHARS[i]]

BATCH_SIZE=512
dataset = torch.utils.data.TensorDataset(X, Y)
data = torch.utils.data.DataLoader(dataset, BATCH_SIZE, shuffle=True)


Num sents: 193075


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self, rnnClass, dictionary_size, embedding_size, num_hiddens, num_classes):
        super().__init__()
        
        self.num_hiddens = num_hiddens
        self.embedding = nn.Embedding(dictionary_size, embedding_size)
        self.hidden = rnnClass(embedding_size, num_hiddens, batch_first=True)
        self.output = nn.Linear(num_hiddens, num_classes)
        
    def forward(self, X):
        out = self.embedding(X)
        _, state = self.hidden(out)
        predictions = self.output(state[0])
        return predictions.squeeze(0)

In [6]:
def sample(preds):
    softmaxed = torch.softmax(preds, 0)
    probas = torch.distributions.multinomial.Multinomial(1, softmaxed).sample()
    return probas.argmax()

def generate_text():
    start_index = random.randint(0, len(text) - MAX_LEN - 1)

    generated = ''
    sentence = text[start_index: start_index + MAX_LEN]
    generated += sentence

    for i in range(MAX_LEN):
        x_pred = torch.zeros((1, MAX_LEN), dtype=int)
        for t, char in enumerate(generated[-MAX_LEN:]):
            x_pred[0, t] = CHAR_TO_INDEX[char]

        preds = model(x_pred.cuda())[0].cpu()
        next_char = INDEX_TO_CHAR[sample(preds)]
        generated = generated + next_char

    print(generated[:MAX_LEN] + '|' + generated[MAX_LEN:])

In [9]:
model = NeuralNetwork(nn.LSTM, len(CHAR_TO_INDEX), 64, 128, len(CHAR_TO_INDEX)).to(dev)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

for ep in range(100):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    model.train()
    for X_b, y_b in tqdm.tqdm_notebook(data):
        X_b, y_b = X_b.to(dev), y_b.to(dev)
        optimizer.zero_grad()
        answers = model.forward(X_b)
        loss = criterion(answers, y_b)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    print("Epoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
    model.eval()
    generate_text()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':



Epoch 0. Time: 3.723, Train loss: 2.174
nated and who therefore await the moment|inns berso prosmencuzy and thiss ensoss 



Epoch 1. Time: 3.588, Train loss: 1.784
nd bad but adherence to traditional cust|iie canged of moad oursen itfenderly eat



Epoch 2. Time: 3.550, Train loss: 1.647
ation and culture one will only succeed |inselien and hirede bexuch upphilst sofe



Epoch 3. Time: 3.661, Train loss: 1.562
lost and is there anything finer than to| have uvidely with the accistious work t


...


Epoch 95. Time: 3.843, Train loss: 0.943
re truth and cry thou deceived one decei|ved which he know that or long at reserv



Epoch 96. Time: 3.951, Train loss: 0.944
valuable services and who is sufficientl|y being pertains preciarament knowledge 



Epoch 97. Time: 3.770, Train loss: 0.941
on the anxiety regarding salvation all n|ecessity of impresendence one s despenta



Epoch 98. Time: 3.804, Train loss: 0.942
 different ages as the peculiarly immora|lity with these conduct still forth and 



Epoch 99. Time: 4.001, Train loss: 0.938
ous deeds of the heroes of chivalrous ro|ped for expression on what man why so wh
